In [1]:
import numpy as np
import pandas as pd

In [12]:

import pandas as pd
import numpy as np

from pandas.core.frame import DataFrame #class
from configuration.configuration import Configuration #class
from typing import List #class
from typing import Set #class

from sklearn.metrics import *
from sklearn.preprocessing import normalize


from datasets.aDataset import ADataset #class
from datasets.datasetML import DatasetML #class
from datasets.datasetRetailrocket import DatasetRetailRocket #class
from datasets.datasetST import DatasetST #class




class ToolMMR():

    def init(self, trainDataset:ADataset):
        if  not isinstance(trainDataset, ADataset):
            raise ValueError("Argument trainDataset isn't type ADataset.")

        if type(trainDataset) is DatasetML:
            cbDataPath = Configuration.cbML1MDataFileWithPathOHE
        elif type(trainDataset) is DatasetRetailRocket:
            return None

        elif type(trainDataset) is DatasetST:
            cbDataPath = Configuration.cbSTDataFileWithPathOHE

        mmrCBFeatures = pd.read_csv(cbDataPath, sep=",", header=0, index_col=0)
        # print(mmrCBFeatures)

        dfCBSim = 1 - pairwise_distances(mmrCBFeatures, metric="cosine")
        self.mmrCBSim: DataFrame = DataFrame(data=dfCBSim, index=mmrCBFeatures.index, columns=mmrCBFeatures.index)
        # print(self.CBSim.shape)

    def _argmax(self, keys, f):
        return max(keys, key=f)

    def _objects_similarity(self, i:int, j:int):
        # print(i,j, self.mmrCBSim.at[i, j])
        try:
            return self.mmrCBSim.at[i, j]
        except:
            #print("miss")
            return 0
    
    def mmr_sorted_with_prefix(self, lambda_:float, results, prefix, length:int):
        selected = pd.Series(dtype=np.float64)
        prefix = set(prefix)
        docs = set(results.index)
        while (len(selected) < len(docs)) and (len(selected) < length):
            remaining = docs - set(selected.index)
            mmr_score = lambda x: lambda_ * results[x] - (1 - lambda_) * max(
                [self._objects_similarity(x, y) for y in set(selected.index).union(prefix) - {x}] or [
                    0])  # TODO: self.mmr_objects_similarity
            next_selected = self._argmax(remaining, mmr_score)
            mmrVal = lambda_ + (lambda_ * results[next_selected] - (1 - lambda_) * max(
                [self._objects_similarity(next_selected, y) for y in set(selected.index).union(prefix) - {next_selected}] or [0]))
            selected = selected.append(pd.Series(mmrVal, index=[next_selected]))
            # selected[next_selected] = mmrVal

        return selected 

In [13]:
from datasets.aDataset import ADataset #class
from datasets.datasetST import DatasetST #class

from datasets.ml.items import Items #class
from datasets.slantour.serials import Serials #class
from datasets.slantour.events import Events #class

dataset:ADataset = DatasetST.readDatasets()


In [14]:
toolMMR = ToolMMR()
toolMMR.init(dataset)

In [15]:
toolMMR

In [16]:
import os
from typing import List
from pandas.core.frame import DataFrame #class

from aggregation.aggrFuzzyDHondt import AggrFuzzyDHondt #class
from aggregation.aggrFuzzyDHondtINF import AggrFuzzyDHondtINF #class

import pandas as pd
from history.aHistory import AHistory #class
from history.historyDF import HistoryDF #class

from userBehaviourDescription.userBehaviourDescription import UserBehaviourDescription #class
from userBehaviourDescription.userBehaviourDescription import observationalLinearProbabilityFnc #function

#from aggregation.toolsDHontNF.penalizationOfResultsByNegImpFeedback.penalUsingReduceRelevance import PenalizationOfResultsByNegImpFeedbackUsingReduceRelevance #class
from aggregation.negImplFeedback.penalUsingReduceRelevance import PenalUsingReduceRelevance #class

from aggregation.negImplFeedback.penalUsingReduceRelevance import penaltyLinear #function

from aggregation.negImplFeedback.penalUsingReduceRelevance import PenalUsingReduceRelevance #class
from aggregation.negImplFeedback.aPenalization import APenalization #class
from aggregation.aggrWeightedAVG import AggrWeightedAVG #class

In [17]:
N = 5

# method results, items=[1,2,4,5,6,7,8,12,32,64,77]
methodsResultDict = {
          "metoda1":pd.Series([0.2,0.1,0.3,0.3,0.1],[6130,5422,5516,4232,2685],name="rating"),
          "metoda2":pd.Series([0.1,0.1,0.2,0.3,0.3],[4232,7511,7522,5516,6130],name="rating"),
          "metoda3":pd.Series([0.3,0.1,0.2,0.3,0.1],[6130,5422,2685,64,7522],name="rating")
          }
# methods parametes
portfolioModelData:List[tuple] = [['metoda1',100], ['metoda2',80], ['metoda3',60]]
portfolioModel:DataFrame = pd.DataFrame(portfolioModelData, columns=["methodID","votes"])
portfolioModel.set_index("methodID", inplace=True)

sumMethodsVotes: float = portfolioModel["votes"].sum()
for methodIdI in portfolioModel.index:
        portfolioModel.loc[methodIdI, "votes"] = portfolioModel.loc[methodIdI, "votes"] / sumMethodsVotes


aggr:AggrWeightedAVG = AggrWeightedAVG(HistoryDF(""), {})
#itemIDs:int = aggr.run(methodsResultDict, methodsParamsDF, N)
#print(itemIDs)

userID:int = 0
itemID:int = 7

itemIDs = aggr.runWithScore(methodsResultDict, portfolioModel, userID, N*5, {})
print(itemIDs)


6130    0.258333
5516    0.225000
4232    0.158333
7522    0.091667
2685    0.091667
64      0.075000
5422    0.066667
7511    0.033333
Name: rating, dtype: float64


In [18]:
previousRecs = [6130,5516,4232,7522,2685]
itemIDs:List[tuple] = aggr.runWithScore(methodsResultDict, portfolioModel, userID, N*5, {})
results = toolMMR.mmr_sorted_with_prefix(0.5, itemIDs, previousRecs, 5)
results

64      0.537500
6130    0.337699
7511    0.285408
2685    0.268989
7522    0.206284
dtype: float64

In [22]:
results.index.tolist()

[64, 6130, 7511, 2685, 7522]